In [1]:
import os,sys,pymysql
import numpy as np
import tensorflow as tf
from gensim.models import word2vec
from sklearn.model_selection import train_test_split


In [52]:
# 文档向量

model = word2vec.Word2Vec.load('Zi_vec.bin')

# 直接横向相加
def wv2dv(t):
    text=""
    for uchar in t:
        if (uchar >= '\u4e00' and uchar <= '\u9fa5'):
            text+=uchar
    lth = len(text)
    dv = np.zeros(100)
    for i in text:
        try:
            dv += model.wv[i]
        except:
            lth-=1
    l = max(1,lth)
    dv = dv/l
    return dv

# 矩阵化
def wv2matrix(t):
    text=""
    matrix = []
    for uchar in t:
        if (uchar >= '\u4e00' and uchar <= '\u9fa5'):
            text+=uchar
    zero = np.zeros(100)
    for i in range(80):
        try:
            matrix.append(model.wv[text[i]])
        except:
            matrix.append(zero)
    return np.array(matrix)

# database part
CONN = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='topic')
cur = CONN.cursor()

def fromdb(topic):
    rs=[]
    for i in range(1,900):
        try:
            sql = "select content from %s where id=%s"%(topic,i)
            cur.execute(sql)
            text = cur.fetchall()[0][0]
            rs.append(text)
        except:
            pass
    return rs

y=wv2matrix('一半残阳下小楼，朱帘斜控软金钩。倚阑无绪不能愁。有个盈盈骑马过，薄妆浅黛亦风流。见人羞涩却回头。')
y.shape

(80, 100)

In [53]:
# 训练集和测试集

topic_dic = {
    'war':      0,#np.array([1,0,0,0,0,]),
    'scene':    1,#np.array([0,1,0,0,0,]),
    'farewell': 2,#np.array([0,0,1,0,0,]),
    'travel':   3,#np.array([0,0,0,1,0,]),
    'love':     4,#np.array([0,0,0,0,1,]),
}

x_raw = []
y_raw = []
topics = ['war','scene','farewell','travel','love']
for index in range(5):
    tag = topic_dic[topics[index]]
    at = fromdb(topics[index])
    for i in at:
        x_raw.append(wv2matrix(i))
        y_raw.append(tag)
x = np.array(x_raw)
y = np.array(y_raw)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=6) # 存在切分不均的问题


In [67]:
model1 = tf.keras.models.Sequential([
    #tf.keras.layers.Conv1D(filters=50, kernel_size=5, strides=1, padding='valid'),
    #tf.keras.layers.MaxPool1D(2, padding='valid'),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.LSTM(100, return_sequences=True),
    #tf.keras.layers.Flatten(input_shape=[80,100]),
    #tf.keras.layers.Dense(300, activation='relu'),
    #tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    #tf.keras.layers.Dense(5, activation='softmax')
    tf.keras.layers.LSTM(5, activation='softmax', return_sequences=False)
])

model1.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

model1.fit(x_train, y_train, epochs=20)
model1.evaluate(x_test,  y_test, verbose=2)

Train on 1765 samples
Epoch 1/20
1765/1765 [==============================] - 8s 5ms/sample - loss: 1.5175 - accuracy: 0.2935
Epoch 2/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.4201 - accuracy: 0.3365
Epoch 3/20
1765/1765 [==============================] - 5s 3ms/sample - loss: 1.3721 - accuracy: 0.4034
Epoch 4/20
1765/1765 [==============================] - 5s 3ms/sample - loss: 1.3242 - accuracy: 0.4051
Epoch 5/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.3238 - accuracy: 0.4079
Epoch 6/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.3185 - accuracy: 0.4238
Epoch 7/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.3361 - accuracy: 0.4550
Epoch 8/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.2740 - accuracy: 0.4499
Epoch 9/20
1765/1765 [==============================] - 6s 3ms/sample - loss: 1.2132 - accuracy: 0.4765
Epoch 10/20
1765/1765 [===================

[1.2411737877160867, 0.5059423]

In [69]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=50, kernel_size=5, strides=1, padding='valid'),
    tf.keras.layers.MaxPool1D(2, padding='valid'),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Flatten(input_shape=[80,100]),
    #tf.keras.layers.Dense(300, activation='relu'),
    #tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model2.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

model2.fit(x_train, y_train, epochs=20)
model2.evaluate(x_test,  y_test, verbose=2)

Train on 1765 samples
Epoch 1/20
1765/1765 [==============================] - 1s 507us/sample - loss: 1.6838 - accuracy: 0.3280
Epoch 2/20
1765/1765 [==============================] - 1s 373us/sample - loss: 1.0412 - accuracy: 0.5745
Epoch 3/20
1765/1765 [==============================] - 1s 385us/sample - loss: 0.6468 - accuracy: 0.7671
Epoch 4/20
1765/1765 [==============================] - 1s 351us/sample - loss: 0.3522 - accuracy: 0.9065
Epoch 5/20
1765/1765 [==============================] - 1s 355us/sample - loss: 0.2534 - accuracy: 0.9337
Epoch 6/20
1765/1765 [==============================] - 1s 355us/sample - loss: 0.2266 - accuracy: 0.9473
Epoch 7/20
1765/1765 [==============================] - 1s 347us/sample - loss: 0.2027 - accuracy: 0.9467
Epoch 8/20
1765/1765 [==============================] - 1s 387us/sample - loss: 0.1784 - accuracy: 0.9479
Epoch 9/20
1765/1765 [==============================] - 1s 345us/sample - loss: 0.1699 - accuracy: 0.9530
Epoch 10/20
1765/1765 [=

[2.0188183833059106, 0.516129]